In [73]:
import cv2
import numpy as np

cap = cv2.VideoCapture("volleyball_video_full_2.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_factor = 0.5  # scale down 50%
new_width = int(width * scale_factor)
new_height = int(height * scale_factor)

out = cv2.VideoWriter("volleyball_tracking.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (new_width, new_height))

background_frames = 50
frame_samples = []

for _ in range(background_frames):
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_samples.append(frame)

background = np.median(np.array(frame_samples), axis=0).astype(np.uint8)

# subtractor = cv2.createBackgroundSubtractorKNN(history=100, dist2Threshold=500, detectShadows=True)
subtractor = cv2.createBackgroundSubtractorMOG2(history=0, detectShadows=True, varThreshold=100)
# subtractor.setNMixtures(5)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # movement_mask = np.any(np.abs(frame_rgb - background) > 20, axis=2).astype(np.uint8) * 255  # Convert to 255 for display
    
    movement_mask = subtractor.apply(frame_rgb)
    
    # detect and remove long lines
    # edges = cv2.Canny(movement_mask, 50, 150, apertureSize=3)
    # line_mask = np.zeros_like(movement_mask)
    # lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=500, minLineLength=150, maxLineGap=100)
    # if lines is not None:
    #     for line in lines:
    #         x1, y1, x2, y2 = line[0]
    #         line_length = np.hypot(x2 - x1, y2 - y1)
    #         if line_length > 1000:  # adjust threshold as needed
    #             cv2.line(movement_mask, (x1, y1), (x2, y2), 0, thickness=20, lineType=cv2.LINE_8)

    
    # movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))
    
    # movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2eq.MORPH_ELLIPSE, (3, 3)))
    # movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))

    # color_mask_value = 0.5 * frame_rgb[:, :, 0] + 0.5 * frame_rgb[:, :, 1] - frame_rgb[:, :, 2]
    # color_mask = ((color_mask_value - np.min(color_mask_value)) / (np.max(color_mask_value) - np.min(color_mask_value)) > 0.77).astype(np.uint8) * 255
    
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    lower_yellow = np.array([80, 0, 0])
    upper_yellow = np.array([110, 255, 255])
    color_mask = cv2.inRange(frame_hsv, lower_yellow, upper_yellow)
    color_mask = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))
    # color_mask = cv2.morphologyEx(color_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))
    

    mask = cv2.bitwise_and(movement_mask, color_mask)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25)))

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ball_mask = np.zeros_like(frame[:, :, 0])  # create a new binary mask for the ball
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        center = (int(x), int(y))
        # cv2.circle(frame, center, 10, (0, 0, 255), -1)
        # cv2.circle(frame, center, int(radius*0.9), (0, 0, 255), -1)
        cv2.circle(ball_mask, center, int(radius*0.9), 255, -1)
        
    ball_only = cv2.bitwise_and(frame, frame, mask=ball_mask)

    frame_resized = cv2.resize(frame, (new_width, new_height))
    ball_only_resized = cv2.resize(ball_only, (new_width, new_height))
    movement_mask_resized = cv2.resize(movement_mask, (new_width, new_height))
    color_mask_resized = cv2.resize(color_mask, (new_width, new_height))
    mask_resized = cv2.resize(mask, (new_width, new_height))
    ball_mask_resized = cv2.resize(ball_mask, (new_width, new_height))

    cv2.imshow("Movement Mask", movement_mask_resized)
    cv2.imshow("Color Mask", color_mask_resized)
    # cv2.imshow("Final Mask (Movement & Color)", mask_resized)
    cv2.imshow("Original", frame_resized)
    cv2.imshow("Ball Mask", ball_mask_resized)
    cv2.imshow("Ball Only", ball_only_resized)
    out.write(frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


In [74]:
import cv2
import numpy as np

cap = cv2.VideoCapture("volleyball_video_full_2.mp4")

knn = cv2.createBackgroundSubtractorKNN(history=0, dist2Threshold=200, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (640, 360))

    fg_mask_knn = knn.apply(frame_resized)

    fg_mask = np.where(fg_mask_knn == 255, 255, 0).astype(np.uint8)

    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel, iterations=2)

    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.4 * dist_transform.max(), 255, 0)
    sure_fg = sure_fg.astype(np.uint8)

    unknown = cv2.subtract(sure_bg, sure_fg)

    _, markers = cv2.connectedComponents(sure_fg)

    markers = markers + 1

    markers[unknown == 255] = 0

    markers = cv2.watershed(frame_resized, markers)

    for marker_id in np.unique(markers):
        if marker_id <= 1:
            continue
        mask = np.uint8(markers == marker_id)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            continue
        c = max(contours, key=cv2.contourArea)
        if cv2.contourArea(c) < 300:
            continue
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(frame_resized, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Foreground Mask (KNN)', fg_mask)
    cv2.imshow('Segmented Players', frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [56]:
import cv2
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture("volleyball_video_full_2.mp4")

# Background subtractor
knn = cv2.createBackgroundSubtractorKNN(history=100, dist2Threshold=500, detectShadows=True)
# knn = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (640, 360))

    # Apply KNN background subtraction
    fg_mask_knn = knn.apply(frame_resized)

    # Remove shadows (127) and clean mask
    fg_mask = np.where(fg_mask_knn == 255, 255, 0).astype(np.uint8)

    # Morphological closing to merge close regions (players close together)
    fg_closed = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 2)))

    # Optional: remove small noise
    # fg_closed = cv2.morphologyEx(fg_closed, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # Find contours
    contours, _ = cv2.findContours(fg_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank mask to draw filled player shapes
    player_mask = np.zeros_like(fg_closed)

    for contour in contours:
        if cv2.contourArea(contour) < 100:
            continue  # filter out small noise
        cv2.drawContours(player_mask, [contour], -1, 255, thickness=cv2.FILLED)

    # Optional: visualize filled shapes on original frame
    result = frame_resized.copy()
    result[player_mask == 255] = [0, 255, 0]  # overlay green for player blobs

    # Show outputs
    cv2.imshow('Foreground Mask (cleaned)', fg_closed)
    cv2.imshow('Foreground Mask (KNN)', fg_mask_knn)
    cv2.imshow('Foreground Mask', fg_mask)
    cv2.imshow('Player Shapes', result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
